In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import VarianceThreshold 
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [2]:
apartment_data = pd.read_csv('apartment_data.csv')

In [3]:
apartment_data.shape

(99188, 9)

# Filtering Outliers

In [4]:
# Calculate the first quartile (Q1) and third quartile (Q3) of the price data
Q1 = apartment_data['price'].quantile(0.25)
Q3 = apartment_data['price'].quantile(0.75)

# Calculate the interquartile range (IQR)
IQR = Q3 - Q1

# Define the lower and upper bounds to filter out outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out prices outside of the IQR range
filtered_apartment_data = apartment_data[(apartment_data['price'] >= lower_bound) & (apartment_data['price'] <= upper_bound)]

print("Original DataFrame:")
display(apartment_data)
print("\nFiltered DataFrame (prices within the IQR range):")
filtered_apartment_data

Original DataFrame:


,bathrooms,bedrooms,price,square_feet,cityname,state,latitude,longitude,region
0,1.0,1.0,2195.0,542,Redondo Beach,CA,33.8520,-118.3759,West
1,1.5,3.0,1250.0,1500,Newport News,VA,37.0867,-76.4941,South
2,2.0,3.0,1395.0,1650,Raleigh,NC,35.8230,-78.6438,South
3,1.0,2.0,1600.0,820,Vacaville,CA,38.3622,-121.9712,West
4,1.0,1.0,975.0,624,Albuquerque,NM,35.1038,-106.6110,West
...,...,...,...,...,...,...,...,...,...
99183,1.0,1.0,780.0,605,Houston,TX,29.6151,-95.1998,South
99184,2.0,2.0,813.0,921,Jacksonville,FL,30.2254,-81.7579,South
99185,1.0,1.0,1325.0,650,San Diego,CA,32.7379,-117.0914,West
99186,1.0,1.0,931.0,701,Huntersville,NC,35.4158,-80.8451,South



Filtered DataFrame (prices within the IQR range):


,bathrooms,bedrooms,price,square_feet,cityname,state,latitude,longitude,region
0,1.0,1.0,2195.0,542,Redondo Beach,CA,33.8520,-118.3759,West
1,1.5,3.0,1250.0,1500,Newport News,VA,37.0867,-76.4941,South
2,2.0,3.0,1395.0,1650,Raleigh,NC,35.8230,-78.6438,South
3,1.0,2.0,1600.0,820,Vacaville,CA,38.3622,-121.9712,West
4,1.0,1.0,975.0,624,Albuquerque,NM,35.1038,-106.6110,West
...,...,...,...,...,...,...,...,...,...
99183,1.0,1.0,780.0,605,Houston,TX,29.6151,-95.1998,South
99184,2.0,2.0,813.0,921,Jacksonville,FL,30.2254,-81.7579,South
99185,1.0,1.0,1325.0,650,San Diego,CA,32.7379,-117.0914,West
99186,1.0,1.0,931.0,701,Huntersville,NC,35.4158,-80.8451,South


In [5]:
exclude_bathrooms = [4.0, 4.5, 5.0, 6.0]

filtered_apartment_data = filtered_apartment_data[~filtered_apartment_data['bathrooms'].isin(exclude_bathrooms)]

In [6]:
filtered_apartment_data['bathrooms'].value_counts()

bathrooms
1.0    55008
2.0    33903
1.5     3032
2.5     1763
3.0      633
3.5      149
Name: count, dtype: int64

In [7]:
exclude_bedrooms = [6.0, 8.0]

filtered_apartment_data = filtered_apartment_data[~filtered_apartment_data['bedrooms'].isin(exclude_bedrooms)]

In [8]:
filtered_apartment_data['bedrooms'].value_counts()

bedrooms
2.0    43441
1.0    39377
3.0     9693
4.0     1190
0.0      628
5.0      146
Name: count, dtype: int64

In [9]:
# Calculate the first quartile (Q1) and third quartile (Q3) of the price data
Q1 = apartment_data['square_feet'].quantile(0.25)
Q3 = apartment_data['square_feet'].quantile(0.75)

# Calculate the interquartile range (IQR)
IQR = Q3 - Q1

# Define the lower and upper bounds to filter out outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out prices outside of the IQR range
filtered_apartment_data = filtered_apartment_data[(filtered_apartment_data['square_feet'] >= lower_bound) & (filtered_apartment_data['square_feet'] <= upper_bound)]

print("\nFiltered DataFrame (square_feet within the IQR range):")
filtered_apartment_data


Filtered DataFrame (square_feet within the IQR range):


,bathrooms,bedrooms,price,square_feet,cityname,state,latitude,longitude,region
0,1.0,1.0,2195.0,542,Redondo Beach,CA,33.8520,-118.3759,West
1,1.5,3.0,1250.0,1500,Newport News,VA,37.0867,-76.4941,South
2,2.0,3.0,1395.0,1650,Raleigh,NC,35.8230,-78.6438,South
3,1.0,2.0,1600.0,820,Vacaville,CA,38.3622,-121.9712,West
4,1.0,1.0,975.0,624,Albuquerque,NM,35.1038,-106.6110,West
...,...,...,...,...,...,...,...,...,...
99183,1.0,1.0,780.0,605,Houston,TX,29.6151,-95.1998,South
99184,2.0,2.0,813.0,921,Jacksonville,FL,30.2254,-81.7579,South
99185,1.0,1.0,1325.0,650,San Diego,CA,32.7379,-117.0914,West
99186,1.0,1.0,931.0,701,Huntersville,NC,35.4158,-80.8451,South


In [10]:
filtered_apartment_data.dtypes

bathrooms      float64
bedrooms       float64
price          float64
square_feet      int64
cityname        object
state           object
latitude       float64
longitude      float64
region          object
dtype: object

# Machine Learning

### Perform Train Test Split

In [11]:
features = apartment_data.drop(columns = ['price', 'cityname', 'state'])
target = apartment_data['price']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=0)

In [13]:
#Separate numerical and categorical columns

X_train_cat= X_train.select_dtypes(include=object)
X_train_num= X_train.select_dtypes(include=np.number)

X_test_cat= X_test.select_dtypes(include=object)
X_test_num= X_test.select_dtypes(include=np.number)

In [14]:
#Transform Numerical Columns (TRAIN)

transformer = MinMaxScaler().fit(X_train_num)
X_train_num_norm = transformer.transform(X_train_num)
X_train_num_scale = pd.DataFrame(X_train_num_norm, index = X_train_num.index, columns=X_train_num.columns).reset_index(drop=True)
X_train_num_scale.head()

,bathrooms,bedrooms,square_feet,latitude,longitude
0,0.000,0.111111,0.051123,0.316856,0.462670
1,0.125,0.222222,0.126545,0.289129,0.465944
2,0.000,0.222222,0.062474,0.276208,0.863663
3,0.125,0.222222,0.060624,0.290905,0.465043
4,0.125,0.222222,0.098209,0.173805,0.847692


In [15]:
#Transform Numerical Columns (TEST)

transformer = MinMaxScaler().fit(X_test_num)
X_test_num_norm = transformer.transform(X_test_num)
X_test_num_scale = pd.DataFrame(X_test_num_norm, index = X_test_num.index, columns=X_test_num.columns).reset_index(drop=True)
X_test_num_scale.head()

,bathrooms,bedrooms,square_feet,latitude,longitude
0,0.0,0.166667,0.038643,0.255318,0.535566
1,0.2,0.500000,0.164552,0.370516,0.478541
2,0.2,0.333333,0.114074,0.362282,0.889514
3,0.2,0.500000,0.146015,0.457310,0.805625
4,0.0,0.166667,0.044204,0.543398,0.785170


In [16]:
#Encode Categorical Columns (TRAIN)

encoded_X_train_cat = pd.get_dummies(X_train_cat)
encoded_X_train_cat.head()

,region_Midwest,region_Northeast,region_South,region_West
56854,False,False,False,True
3997,False,False,False,True
85239,False,False,True,False
82697,False,False,False,True
33681,False,False,True,False


In [17]:
#Encode Categorical Columns (TEST)

encoded_X_test_cat = pd.get_dummies(X_test_cat)
encoded_X_test_cat.head()

,region_Midwest,region_Northeast,region_South,region_West
54841,False,False,False,True
44135,False,False,False,True
23266,False,False,True,False
3108,True,False,False,False
3780,True,False,False,False


## KNN Regressor

In [18]:
from sklearn.neighbors import KNeighborsRegressor

In [19]:
knn = KNeighborsRegressor(n_neighbors=10)

knn.fit(X_train_num_scale, y_train)

knn.score(X_test_num_scale, y_test)

-0.0728101559263703

In [41]:
knn = KNeighborsRegressor(n_neighbors=10)

knn.fit(X_train_num, y_train)

knn.score(X_test_num, y_test)

0.49562357642959765

In [20]:
#From 49.5%

## Linear Regression

In [21]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()

In [22]:
lin_reg.fit(X_train_num_scale, y_train)

LinearRegression()

In [43]:
lin_reg.fit(X_train_num, y_train)

LinearRegression()

In [23]:
pred = lin_reg.predict(X_test_num_scale)

print("MAE", mean_absolute_error(y_test, pred))
print("RMSE", mean_squared_error(y_test, pred, squared=False))
print("R2 score", lin_reg.score(X_test_num, y_test))

MAE 673.3494804582056
RMSE 905.8278967393768
R2 score -0.12691797732919596


In [44]:
pred = lin_reg.predict(X_test_num)

print("MAE", mean_absolute_error(y_test, pred))
print("RMSE", mean_squared_error(y_test, pred, squared=False))
print("R2 score", lin_reg.score(X_test_num, y_test))

MAE 493.99431006698296
RMSE 767.4198681935658
R2 score 0.1911518406509063


## Bagging and Pasting

In [24]:
bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=20),
                               n_estimators=100,
                               max_samples = 1000)

In [25]:
bagging_reg.fit(X_train_num_scale, y_train)

BaggingRegressor(estimator=DecisionTreeRegressor(max_depth=20),
                 max_samples=1000, n_estimators=100)

In [45]:
bagging_reg.fit(X_train_num, y_train)

BaggingRegressor(estimator=DecisionTreeRegressor(max_depth=20),
                 max_samples=1000, n_estimators=100)

In [26]:
pred = bagging_reg.predict(X_test_num_scale)

print("MAE", mean_absolute_error(y_test, pred))
print("RMSE", mean_squared_error(y_test, pred, squared=False))
print("R2 score", lin_reg.score(X_test_num_scale, y_test))

MAE 550.5188420946688
RMSE 837.7620277248777
R2 score 0.03607716760878232


In [46]:
pred = bagging_reg.predict(X_test_num)

print("MAE", mean_absolute_error(y_test, pred))
print("RMSE", mean_squared_error(y_test, pred, squared=False))
print("R2 score", lin_reg.score(X_test_num, y_test))

MAE 273.28238322972044
RMSE 524.8670283774038
R2 score 0.6216449739816411


## Random Forest

In [27]:
forest = RandomForestRegressor(n_estimators=100,
                             max_depth=20)

In [28]:
forest.fit(X_train_num_scale, y_train)

RandomForestRegressor(max_depth=20)

In [47]:
forest.fit(X_train_num, y_train)

RandomForestRegressor(max_depth=20)

In [29]:
pred = forest.predict(X_test_num_scale)

print("MAE", mean_absolute_error(y_test, pred))
print("RMSE", mean_squared_error(y_test, pred, squared=False))
print("R2 score", forest.score(X_test_num_scale, y_test))

MAE 604.3543632644933
RMSE 1112.9139809534574
R2 score -0.701077579798296


In [48]:
pred = forest.predict(X_test_num)

print("MAE", mean_absolute_error(y_test, pred))
print("RMSE", mean_squared_error(y_test, pred, squared=False))
print("R2 score", lin_reg.score(X_test_num, y_test))

MAE 181.58032149294743
RMSE 417.7403483921684
R2 score 0.1911518406509063


## AdaBoost

In [30]:
ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=20),
                            n_estimators=100)

In [31]:
ada_reg.fit(X_train_num_scale, y_train)

AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=20),
                  n_estimators=100)

In [49]:
ada_reg.fit(X_train_num, y_train)

AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=20),
                  n_estimators=100)

In [32]:
pred = ada_reg.predict(X_test_num_scale)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", ada_reg.score(X_test_num_scale, y_test))

MAE 556.1887628714104
RMSE 981.4623606588491
R2 score -0.32296459553460743


In [50]:
pred = ada_reg.predict(X_test_num_scale)

print("MAE", mean_absolute_error(y_test, pred))
print("RMSE", mean_squared_error(y_test, pred, squared=False))
print("R2 score", lin_reg.score(X_test_num, y_test))

MAE 556.3602686762779
RMSE 854.3583644972377
R2 score 0.1911518406509063


## Gradient Boosting

In [33]:
gb_reg = GradientBoostingRegressor(max_depth=20,
                                   n_estimators=100)

In [34]:
gb_reg.fit(X_train_num_scale, y_train)

GradientBoostingRegressor(max_depth=20)

In [51]:
gb_reg.fit(X_train_num, y_train)

GradientBoostingRegressor(max_depth=20)

In [35]:
pred = gb_reg.predict(X_test_num_scale)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", gb_reg.score(X_test_num_scale, y_test))

MAE 645.5115213084523
RMSE 1313.7646997180532
R2 score -1.3704789377508355


In [52]:
pred = gb_reg.predict(X_test_num_scale)

print("MAE", mean_absolute_error(y_test, pred))
print("RMSE", mean_squared_error(y_test, pred, squared=False))
print("R2 score", lin_reg.score(X_test_num, y_test))

MAE 851.1862364857825
RMSE 1054.1607189621814
R2 score 0.1911518406509063


## Decision Tree

In [36]:
tree = DecisionTreeRegressor(max_depth=10)

In [37]:
tree.fit(X_train_num_scale, y_train)

DecisionTreeRegressor(max_depth=10)

In [53]:
tree.fit(X_train_num, y_train)

DecisionTreeRegressor(max_depth=10)

In [38]:
pred = tree.predict(X_test_num_scale)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", tree.score(X_test_num_scale, y_test))

MAE 610.0328923047938
RMSE 1277.6299474332443
R2 score -1.2418734315150077


In [54]:
pred = tree.predict(X_test_num_scale)

print("MAE", mean_absolute_error(y_test, pred))
print("RMSE", mean_squared_error(y_test, pred, squared=False))
print("R2 score", lin_reg.score(X_test_num, y_test))

MAE 2389.7767421111
RMSE 2497.566589009985
R2 score 0.1911518406509063


In [39]:
tree_importance = {feature : importance for feature, importance in zip(X_train_num_scale.columns, tree.feature_importances_)}
tree_importance           

{'bathrooms': 0.1681135267459292,
 'bedrooms': 0.025516962350658003,
 'square_feet': 0.23852818580305085,
 'latitude': 0.17878487116988548,
 'longitude': 0.3890564539304765}